![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🔗 Section 3: Memory-Enhanced RAG and Agents

**⏱️ Estimated Time:** 60-75 minutes

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Build** a memory-enhanced RAG system that combines all four context types
2. **Demonstrate** the benefits of memory for natural conversations
3. **Convert** a simple RAG system into a LangGraph agent
4. **Prepare** for Section 4 (adding tools and advanced agent capabilities)

---

## 🔗 Bridge from Previous Notebooks

### **What You've Learned:**

**Section 1:** Four Context Types
- System Context (static instructions)
- User Context (profile, preferences)
- Conversation Context (enabled by working memory)
- Retrieved Context (RAG results)

**Section 2:** RAG Fundamentals
- Semantic search with vector embeddings
- Context assembly
- LLM generation

**Section 3 (Notebook 1):** Memory Fundamentals
- Working memory for conversation continuity
- Long-term memory for persistent knowledge
- Memory types (semantic, episodic, message)
- Memory lifecycle and persistence

### **What We'll Build:**

**Part 1:** Memory-Enhanced RAG
- Integrate working memory + long-term memory + RAG
- Show clear before/after comparisons
- Demonstrate benefits of memory systems

**Part 2:** LangGraph Agent (Separate Notebook)
- Convert memory-enhanced RAG to LangGraph agent
- Add state management and control flow
- Prepare for Section 4 (tools and advanced capabilities)

---

## 📊 The Complete Picture

### **Memory-Enhanced RAG Flow:**

```
User Query
    ↓
1. Load Working Memory (conversation history)
2. Search Long-term Memory (user preferences, facts)
3. RAG Search (relevant courses)
4. Assemble Context (System + User + Conversation + Retrieved)
5. Generate Response
6. Save Working Memory (updated conversation)
```

### **All Four Context Types Working Together:**

| Context Type | Source | Purpose |
|-------------|--------|---------|
| **System** | Static prompt | Role, instructions, guidelines |
| **User** | Profile + Long-term Memory | Personalization, preferences |
| **Conversation** | Working Memory | Reference resolution, continuity |
| **Retrieved** | RAG Search | Relevant courses, information |

**💡 Key Insight:** Memory transforms stateless RAG into stateful, personalized conversations.

---

## 📦 Setup

### **What We're Importing:**

- **Section 2 components** - `redis_config`, `CourseManager`, models
- **Agent Memory Server client** - `MemoryAPIClient` for memory operations
- **LangChain** - `ChatOpenAI` for LLM interaction

### **Why:**

- Build on Section 2's RAG foundation
- Add memory capabilities without rewriting everything
- Use production-ready memory infrastructure


In [1]:
# Setup: Import components
import os
import sys
import asyncio
from typing import List, Dict, Any, Optional
from datetime import datetime
from dotenv import load_dotenv

# Load environment
load_dotenv()
sys.path.append('../../reference-agent')

# Import Section 2 components
from redis_context_course.redis_config import redis_config
from redis_context_course.course_manager import CourseManager
from redis_context_course.models import (
    Course, StudentProfile, DifficultyLevel,
    CourseFormat, Semester
)

# Import LangChain
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Import Agent Memory Server client
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import WorkingMemory, MemoryMessage, ClientMemoryRecord
    MEMORY_SERVER_AVAILABLE = True
    print("✅ Agent Memory Server client available")
except ImportError:
    MEMORY_SERVER_AVAILABLE = False
    print("⚠️  Agent Memory Server not available")
    print("📝 Install with: pip install agent-memory-client")
    print("🚀 Start server: See reference-agent/README.md")

# Verify environment
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found. Please set in .env file.")
else:
    print("✅ OPENAI_API_KEY found")

print(f"\n🔧 Environment Setup:")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"   REDIS_URL: {os.getenv('REDIS_URL', 'redis://localhost:6379')}")
print(f"   AGENT_MEMORY_URL: {os.getenv('AGENT_MEMORY_URL', 'http://localhost:8088')}")
print(f"   Memory Server: {'✓ Available' if MEMORY_SERVER_AVAILABLE else '✗ Not available'}")


✅ Agent Memory Server client available
✅ OPENAI_API_KEY found

🔧 Environment Setup:
   OPENAI_API_KEY: ✓ Set
   REDIS_URL: redis://localhost:6379
   AGENT_MEMORY_URL: http://localhost:8088
   Memory Server: ✓ Available


### 🎯 What We Just Did

**Successfully Imported:**
- ✅ **Section 2 RAG components** - `redis_config`, `CourseManager`, models
- ✅ **Agent Memory Server client** - Production-ready memory system
- ✅ **Environment verified** - OpenAI API key, Redis, Memory Server

**Why This Matters:**
- We're **building on Section 2's foundation** (not starting from scratch)
- **Agent Memory Server** provides scalable, persistent memory
- **Same Redis University domain** for consistency

---

## 🔧 Initialize Components


In [2]:
# Initialize components
course_manager = CourseManager()
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# Initialize Memory Client
if MEMORY_SERVER_AVAILABLE:
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8088"),
        default_namespace="redis_university"
    )
    memory_client = MemoryAPIClient(config=config)
    print("🧠 Memory Client Initialized")
    print(f"   Base URL: {config.base_url}")
    print(f"   Namespace: {config.default_namespace}")
else:
    memory_client = None
    print("⚠️  Running without Memory Server (limited functionality)")

# Create a sample student profile (reusing Section 2 pattern)
sarah = StudentProfile(
    name="Sarah Chen",
    email="sarah.chen@university.edu",
    major="Computer Science",
    year=2,
    interests=["machine learning", "data science", "algorithms"],
    completed_courses=["CS101", "CS201"],
    current_courses=["MATH301"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE
)

print(f"\n👤 Student Profile: {sarah.name}")
print(f"   Major: {sarah.major}")
print(f"   Interests: {', '.join(sarah.interests)}")


10:27:08 redisvl.index.index INFO   Index already exists, not overwriting.


🧠 Memory Client Initialized
   Base URL: http://localhost:8088
   Namespace: redis_university

👤 Student Profile: Sarah Chen
   Major: Computer Science
   Interests: machine learning, data science, algorithms


### 💡 Key Insight

We're reusing:
- ✅ **Same `CourseManager`** from Section 2
- ✅ **Same `StudentProfile`** model
- ✅ **Same Redis configuration**

We're adding:
- ✨ **Memory Client** for conversation history
- ✨ **Working Memory** for session context
- ✨ **Long-term Memory** for persistent knowledge

---

## 📚 Part 1: Memory-Enhanced RAG

### **Goal:** Build a simple, inline memory-enhanced RAG system that demonstrates the benefits of memory.

### **Approach:**
- Start with Section 2's stateless RAG
- Add working memory for conversation continuity
- Add long-term memory for personalization
- Show clear before/after comparisons

---

## 🚫 Before: Stateless RAG (Section 2 Approach)

Let's first recall how Section 2's stateless RAG worked.


In [3]:
# Stateless RAG (Section 2 approach)
async def stateless_rag_query(user_query: str, student_profile: StudentProfile, top_k: int = 3) -> str:
    """
    Section 2 stateless RAG approach.

    Problems:
    - No conversation history
    - Can't resolve references ("it", "that course")
    - Each query is independent
    """

    # Step 1: Search courses
    courses = await course_manager.search_courses(user_query, limit=top_k)

    # Step 2: Assemble context (System + User + Retrieved only)
    system_prompt = "You are a helpful Redis University course advisor."

    user_context = f"""Student: {student_profile.name}
Major: {student_profile.major}
Interests: {', '.join(student_profile.interests)}
Completed: {', '.join(student_profile.completed_courses)}"""

    retrieved_context = "Relevant Courses:\n"
    for i, course in enumerate(courses, 1):
        retrieved_context += f"{i}. {course.course_code}: {course.title}\n"

    # Step 3: Generate response
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"{user_context}\n\n{retrieved_context}\n\nQuery: {user_query}")
    ]

    response = llm.invoke(messages).content

    # ❌ No conversation history stored
    # ❌ Next query won't remember this interaction

    return response

# Test stateless RAG
print("=" * 80)
print("🚫 STATELESS RAG DEMO")
print("=" * 80)

query_1 = "I'm interested in machine learning courses"
print(f"\n👤 User: {query_1}")
response_1 = await stateless_rag_query(query_1, sarah)
print(f"\n🤖 Agent: {response_1}")

# Try a follow-up with pronoun reference
query_2 = "What are the prerequisites for the first one?"
print(f"\n\n👤 User: {query_2}")
response_2 = await stateless_rag_query(query_2, sarah)
print(f"\n🤖 Agent: {response_2}")
print("\n❌ Agent can't resolve 'the first one' - no conversation history!")


🚫 STATELESS RAG DEMO

👤 User: I'm interested in machine learning courses


10:27:09 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


10:27:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: Hi Sarah! It's great to hear about your interest in machine learning. Since you've already completed CS101 and CS201, you have a solid foundation in computer science, which will be beneficial as you dive into machine learning.

Here are some course recommendations that align with your interests:

1. **CS007: Machine Learning** - This course is a perfect fit for you as it focuses on the fundamentals of machine learning, including supervised and unsupervised learning techniques, model evaluation, and practical applications. It will build on your existing knowledge and introduce you to key machine learning concepts.

2. **MATH022: Linear Algebra** - Linear algebra is a crucial mathematical foundation for understanding machine learning algorithms. This course will cover essential topics such as vector spaces, matrices, and eigenvalues, which are frequently used in machine learning.

3. **MATH024: Linear Algebra** - If MATH022 is not available or if you're looking for a different 

10:27:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


10:27:19 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: For the course MATH028: Calculus I, the prerequisites typically include a solid understanding of high school algebra and trigonometry. Some institutions may require a placement test to ensure readiness for calculus. However, specific prerequisites can vary by institution, so it's always a good idea to check the course catalog or contact the mathematics department at your university for the most accurate information.

❌ Agent can't resolve 'the first one' - no conversation history!




### 🎯 What Just Happened?

**Query 1:** "I'm interested in machine learning courses"
- ✅ Works fine - searches and returns ML courses

**Query 2:** "What are the prerequisites for **the first one**?"
- ❌ **Fails** - Agent doesn't know what "the first one" refers to
- ❌ No conversation history stored
- ❌ Each query is completely independent

**The Problem:** Natural conversation requires context from previous turns.

---

## ✅ After: Memory-Enhanced RAG

Now let's add memory to enable natural conversations.

### **Step 1: Load Working Memory**

Working memory stores conversation history for the current session.


In [4]:
# Step 1: Load working memory
async def load_working_memory(session_id: str, student_id: str):
    """Load conversation history from working memory"""

    if not MEMORY_SERVER_AVAILABLE:
        return None

    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    return working_memory

# Test loading working memory
session_id = "demo_session_001"
student_id = sarah.email.split('@')[0]

working_memory = await load_working_memory(session_id, student_id)

if working_memory:
    print(f"✅ Loaded working memory for session: {session_id}")
    print(f"   Messages: {len(working_memory.messages)}")
else:
    print("⚠️  Memory Server not available")


10:27:19 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/demo_session_001?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 404 Not Found"


10:27:19 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/demo_session_001?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 500 Internal Server Error"


MemoryServerError: HTTP 500: Internal Server Error

### 🎯 What We Just Did

**Loaded Working Memory:**
- Created or retrieved conversation history for this session
- Session ID: `demo_session_001` (unique per conversation)
- User ID: `sarah_chen` (from student email)

**Why This Matters:**
- Working memory persists across turns in the same session
- Enables reference resolution ("it", "that course", "the first one")
- Conversation context is maintained

---

### **Step 2: Search Long-term Memory**

Long-term memory stores persistent facts and preferences across sessions.


In [ ]:
# Step 2: Search long-term memory
async def search_longterm_memory(query: str, student_id: str, limit: int = 5):
    """Search long-term memory for relevant facts"""

    if not MEMORY_SERVER_AVAILABLE:
        return []

    results = await memory_client.search_long_term_memory(
        text=query,
        user_id=student_id,
        limit=limit
    )

    return [m.text for m in results.memories] if results.memories else []

# Test searching long-term memory
query = "What does the student prefer?"
memories = await search_longterm_memory(query, student_id)

print(f"🔍 Query: '{query}'")
print(f"📚 Found {len(memories)} relevant memories:")
for i, memory in enumerate(memories, 1):
    print(f"   {i}. {memory}")


### 🎯 What We Just Did

**Searched Long-term Memory:**
- Used semantic search to find relevant facts
- Query: "What does the student prefer?"
- Results: Memories about preferences, goals, academic info

**Why This Matters:**
- Long-term memory enables personalization
- Facts persist across sessions (days, weeks, months)
- Semantic search finds relevant memories without exact keyword matching

---

### **Step 3: Assemble All Four Context Types**

Now let's combine everything: System + User + Conversation + Retrieved.


In [ ]:
# Step 3: Assemble all four context types
async def assemble_context(
    user_query: str,
    student_profile: StudentProfile,
    session_id: str,
    top_k: int = 3
):
    """
    Assemble all four context types.

    Returns:
        - system_prompt: System Context
        - user_context: User Context (profile + long-term memories)
        - conversation_messages: Conversation Context (working memory)
        - retrieved_context: Retrieved Context (RAG results)
    """

    student_id = student_profile.email.split('@')[0]

    # 1. System Context (static)
    system_prompt = """You are a Redis University course advisor.

Your role:
- Help students find and enroll in courses
- Provide personalized recommendations
- Answer questions about courses, prerequisites, schedules

Guidelines:
- Use conversation history to resolve references ("it", "that course")
- Use long-term memories to personalize recommendations
- Be helpful, supportive, and encouraging"""

    # 2. User Context (profile + long-term memories)
    user_context = f"""Student Profile:
- Name: {student_profile.name}
- Major: {student_profile.major}
- Year: {student_profile.year}
- Interests: {', '.join(student_profile.interests)}
- Completed: {', '.join(student_profile.completed_courses)}
- Current: {', '.join(student_profile.current_courses)}
- Preferred Format: {student_profile.preferred_format.value}
- Preferred Difficulty: {student_profile.preferred_difficulty.value}"""

    # Search long-term memory
    longterm_memories = await search_longterm_memory(user_query, student_id)
    if longterm_memories:
        user_context += f"\n\nLong-term Memories:\n" + "\n".join([f"- {m}" for m in longterm_memories])

    # 3. Conversation Context (working memory)
    working_memory = await load_working_memory(session_id, student_id)
    conversation_messages = []
    if working_memory:
        for msg in working_memory.messages:
            if msg.role == "user":
                conversation_messages.append(HumanMessage(content=msg.content))
            elif msg.role == "assistant":
                conversation_messages.append(AIMessage(content=msg.content))


    # 4. Retrieved Context (RAG)
    courses = await course_manager.search_courses(user_query, limit=top_k)
    retrieved_context = "Relevant Courses:\n"
    for i, course in enumerate(courses, 1):
        retrieved_context += f"\n{i}. {course.course_code}: {course.title}"
        retrieved_context += f"\n   Description: {course.description}"
        retrieved_context += f"\n   Difficulty: {course.difficulty_level.value}"
        retrieved_context += f"\n   Format: {course.format.value}"
        if course.prerequisites:
            prereqs = [p.course_code for p in course.prerequisites]
            retrieved_context += f"\n   Prerequisites: {', '.join(prereqs)}"

    return system_prompt, user_context, conversation_messages, retrieved_context

# Test assembling context
system_prompt, user_context, conversation_messages, retrieved_context = await assemble_context(
    user_query="machine learning courses",
    student_profile=sarah,
    session_id=session_id,
    top_k=3
)

print("=" * 80)
print("📊 ASSEMBLED CONTEXT")
print("=" * 80)
print(f"\n1️⃣ System Context: {len(system_prompt)} chars")
print(f"2️⃣ User Context: {len(user_context)} chars")
print(f"3️⃣ Conversation Context: {len(conversation_messages)} messages")
print(f"4️⃣ Retrieved Context: {len(retrieved_context)} chars")


### 🎯 What We Just Did

**Assembled All Four Context Types:**

1. **System Context** - Role, instructions, guidelines (static)
2. **User Context** - Profile + long-term memories (dynamic, user-specific)
3. **Conversation Context** - Working memory messages (dynamic, session-specific)
4. **Retrieved Context** - RAG search results (dynamic, query-specific)

**Why This Matters:**
- All four context types from Section 1 are now working together
- System knows WHO the user is (User Context)
- System knows WHAT was discussed (Conversation Context)
- System knows WHAT's relevant (Retrieved Context)
- System knows HOW to behave (System Context)

---

### **Step 4: Generate Response and Save Memory**

Now let's generate a response and save the updated conversation.


In [ ]:
# Step 4: Generate response and save memory
async def generate_and_save(
    user_query: str,
    student_profile: StudentProfile,
    session_id: str,
    top_k: int = 3
) -> str:
    """Generate response and save to working memory"""

    if not MEMORY_SERVER_AVAILABLE:
        # Fallback to stateless RAG
        return await stateless_rag_query(user_query, student_profile, top_k)

    student_id = student_profile.email.split('@')[0]

    # Assemble context
    system_prompt, user_context, conversation_messages, retrieved_context = await assemble_context(
        user_query, student_profile, session_id, top_k
    )

    # Build messages
    messages = [SystemMessage(content=system_prompt)]
    messages.extend(conversation_messages)  # Add conversation history
    messages.append(HumanMessage(content=f"{user_context}\n\n{retrieved_context}\n\nQuery: {user_query}"))

    # Generate response
    response = llm.invoke(messages).content

    # Save to working memory
    working_memory = await load_working_memory(session_id, student_id)
    if working_memory:
        working_memory.messages.extend([
            MemoryMessage(role="user", content=user_query),
            MemoryMessage(role="assistant", content=response)
        ])
        await memory_client.put_working_memory(
            session_id=session_id,
            memory=working_memory,
            user_id=student_id,
            model_name="gpt-4o"
        )

    return response

# Test generating and saving
query = "I'm interested in machine learning courses"
response = await generate_and_save(query, sarah, session_id)

print(f"👤 User: {query}")
print(f"\n🤖 Agent: {response}")
print(f"\n✅ Conversation saved to working memory")


### 🎯 What We Just Did

**Generated Response:**
- Assembled all four context types
- Built message list with conversation history
- Generated response using LLM
- **Saved updated conversation to working memory**

**Why This Matters:**
- Next query will have access to this conversation
- Reference resolution will work ("it", "that course")
- Conversation continuity is maintained

---

## 🧪 Complete Demo: Memory-Enhanced RAG

Now let's test the complete system with a multi-turn conversation.


In [ ]:
# Complete memory-enhanced RAG demo
async def memory_enhanced_rag_demo():
    """Demonstrate complete memory-enhanced RAG system"""

    demo_session_id = "complete_demo_session"

    print("=" * 80)
    print("🧪 MEMORY-ENHANCED RAG DEMO")
    print("=" * 80)
    print(f"\n👤 Student: {sarah.name}")
    print(f"📧 Session: {demo_session_id}")

    # Turn 1: Initial query
    print("\n" + "=" * 80)
    print("📍 TURN 1: Initial Query")
    print("=" * 80)

    query_1 = "I'm interested in machine learning courses"
    print(f"\n👤 User: {query_1}")

    response_1 = await generate_and_save(query_1, sarah, demo_session_id)
    print(f"\n🤖 Agent: {response_1}")

    # Turn 2: Follow-up with pronoun reference
    print("\n" + "=" * 80)
    print("📍 TURN 2: Follow-up with Pronoun Reference")
    print("=" * 80)

    query_2 = "What are the prerequisites for the first one?"
    print(f"\n👤 User: {query_2}")

    response_2 = await generate_and_save(query_2, sarah, demo_session_id)
    print(f"\n🤖 Agent: {response_2}")
    print("\n✅ Agent resolved 'the first one' using conversation history!")


    # Turn 3: Another follow-up
    print("\n" + "=" * 80)
    print("📍 TURN 3: Another Follow-up")
    print("=" * 80)

    query_3 = "Do I meet those prerequisites?"
    print(f"\n👤 User: {query_3}")

    response_3 = await generate_and_save(query_3, sarah, demo_session_id)
    print(f"\n🤖 Agent: {response_3}")
    print("\n✅ Agent resolved 'those prerequisites' and checked student's transcript!")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Memory-enhanced RAG enables natural conversations!")
    print("=" * 80)

# Run the complete demo
await memory_enhanced_rag_demo()


### 🎯 What Just Happened?

**Turn 1:** "I'm interested in machine learning courses"
- System searches courses
- Finds ML-related courses
- Responds with recommendations
- **Saves conversation to working memory**

**Turn 2:** "What are the prerequisites for **the first one**?"
- System loads working memory (Turn 1)
- Resolves "the first one" → first course mentioned in Turn 1
- Responds with prerequisites
- **Saves updated conversation**

**Turn 3:** "Do I meet **those prerequisites**?"
- System loads working memory (Turns 1-2)
- Resolves "those prerequisites" → prerequisites from Turn 2
- Checks student's completed courses (from profile)
- Responds with personalized answer
- **Saves updated conversation**

**💡 Key Insight:** Memory + RAG = **Natural, stateful, personalized conversations**

---

## 📊 Before vs. After Comparison

Let's visualize the difference between stateless and memory-enhanced RAG.

### **Stateless RAG (Section 2):**

```
Query 1: "I'm interested in ML courses"
  → ✅ Works (searches and returns courses)

Query 2: "What are the prerequisites for the first one?"
  → ❌ Fails (no conversation history)
  → Agent: "Which course are you referring to?"
```

**Problems:**
- ❌ No conversation continuity
- ❌ Can't resolve references
- ❌ Each query is independent
- ❌ Poor user experience

### **Memory-Enhanced RAG (This Notebook):**

```
Query 1: "I'm interested in ML courses"
  → ✅ Works (searches and returns courses)
  → Saves to working memory

Query 2: "What are the prerequisites for the first one?"
  → ✅ Works (loads conversation history)
  → Resolves "the first one" → first course from Query 1
  → Responds with prerequisites
  → Saves updated conversation

Query 3: "Do I meet those prerequisites?"
  → ✅ Works (loads conversation history)
  → Resolves "those prerequisites" → prerequisites from Query 2
  → Checks student transcript
  → Responds with personalized answer
```

**Benefits:**
- ✅ Conversation continuity
- ✅ Reference resolution
- ✅ Personalization
- ✅ Natural user experience

---

## 🎓 Key Takeaways

### **1. Memory Transforms RAG**

**Without Memory (Section 2):**
- Stateless queries
- No conversation continuity
- Limited to 3 context types (System, User, Retrieved)

**With Memory (This Notebook):**
- Stateful conversations
- Reference resolution
- All 4 context types (System, User, Conversation, Retrieved)

### **2. Two Types of Memory Work Together**

**Working Memory:**
- Session-scoped conversation history
- Enables reference resolution
- TTL-based (expires after 24 hours)

**Long-term Memory:**
- User-scoped persistent facts
- Enables personalization
- Persists indefinitely

### **3. Simple, Inline Approach**

**What We Built:**
- Small, focused functions
- Inline code (no large classes)
- Progressive learning
- Clear demonstrations

**Why This Matters:**
- Easy to understand
- Easy to modify
- Easy to extend
- Foundation for LangGraph agents (Part 2)

### **4. All Four Context Types**

**System Context:** Role, instructions, guidelines
**User Context:** Profile + long-term memories
**Conversation Context:** Working memory
**Retrieved Context:** RAG results

**Together:** Natural, stateful, personalized conversations

---

## 🚀 What's Next?

### **Part 2: Converting to LangGraph Agent (Separate Notebook)**

In the next notebook (`03_langgraph_agent_conversion.ipynb`), we'll:

1. **Convert** memory-enhanced RAG to LangGraph agent
2. **Add** state management and control flow
3. **Prepare** for Section 4 (tools and advanced capabilities)
4. **Build** a foundation for production-ready agents

**Why LangGraph?**
- Better state management
- More control over agent flow
- Easier to add tools (Section 4)
- Production-ready architecture

### **Section 4: Tools and Advanced Agents**

After completing Part 2, you'll be ready for Section 4:
- Adding tools (course enrollment, schedule management)
- Multi-step reasoning
- Error handling and recovery
- Production deployment

---

## 🏋️ Practice Exercises

### **Exercise 1: Add Personalization**

Modify the system to use long-term memories for personalization:

1. Store student preferences in long-term memory
2. Search long-term memory in `assemble_context()`
3. Use memories to personalize recommendations

**Hint:** Use `memory_client.create_long_term_memory()` and `memory_client.search_long_term_memory()`

### **Exercise 2: Add Error Handling**

Add error handling for memory operations:

1. Handle case when Memory Server is unavailable
2. Fallback to stateless RAG
3. Log warnings appropriately

**Hint:** Check `MEMORY_SERVER_AVAILABLE` flag

### **Exercise 3: Add Conversation Summary**

Add a function to summarize the conversation:

1. Load working memory
2. Extract key points from conversation
3. Display summary to user

**Hint:** Use LLM to generate summary from conversation history

---

## 📝 Summary

### **What You Learned:**

1. ✅ **Built** memory-enhanced RAG system
2. ✅ **Integrated** all four context types
3. ✅ **Demonstrated** benefits of memory
4. ✅ **Prepared** for LangGraph conversion

### **Key Concepts:**

- **Working Memory** - Session-scoped conversation history
- **Long-term Memory** - User-scoped persistent facts
- **Context Assembly** - Combining all four context types
- **Reference Resolution** - Resolving pronouns and references
- **Stateful Conversations** - Natural, continuous dialogue

### **Next Steps:**

1. Complete practice exercises
2. Experiment with different queries
3. Move to Part 2 (LangGraph agent conversion)
4. Prepare for Section 4 (tools and advanced agents)

**🎉 Congratulations!** You've built a complete memory-enhanced RAG system!

---

## 🔗 Resources

- **Section 1:** Four Context Types
- **Section 2:** RAG Fundamentals
- **Section 3 (Notebook 1):** Memory Fundamentals
- **Section 3 (Notebook 3):** LangGraph Agent Conversion (Next)
- **Section 4:** Tools and Advanced Agents

**Agent Memory Server:**
- GitHub: `reference-agent/`
- Documentation: See README.md
- API Client: `agent-memory-client`

**LangChain:**
- Documentation: https://python.langchain.com/
- LangGraph: https://langchain-ai.github.io/langgraph/

---

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

**Redis University - Context Engineering Course**
